# Step 5
The code in this notebook uses some support functions defined in the file `../support.js` in the root of this repository.

## 1 Upload images
We have provided some test images in the directory `data/images` in this repository. Before we can run this notebook we will need to upload them to our S3 bucket that we just created. To do this run the code below:

In [ ]:
exec = require('child_process').exec

const ls = exec('aws s3 sync ../data/images/ s3://$MY_BUCKET_NAME/images', function (error, stdout, stderr) {
  if (error) {
    console.log(error)
  }
  console.log(stdout)
  console.log(stderr)
});

ls.on('exit', function (code) {
  console.log('Child process exited with exit code ' + code)
})
$$.clear()

Note that we are cheating here a bit! The AWS SDK has a `putObject` function but this would require us to loop over the image files and call put object for each one. Instead we are creating a child process inside our container that calls the AWS CLI to run an s3 `sync` command. This will copy the files to the bucket for us.

You should be able to see these images in your newly created bucket by opening the AWS web console and viewing the bucket.

## 2 Label Detection
First up let's use Rekognition to figure out what it thinks an image contains. This process is called label detection. In the code below we will ask Rekognition to find a maximum of ten labels in the image and only return labels that it has a greater than 80% confidence in.

![Eric the Cat](../data/images/cat-pet-animal-domestic-104827.jpeg "Eric")

In [ ]:
// load the AWS SDK
AWS = require('aws-sdk')
support = require('/home/dev/support')

// create a Rekogntion object to call
rek = new AWS.Rekognition()

// configure parameters
imageParams = {
  Image: {
    S3Object: {
      Bucket: process.env.MY_BUCKET_NAME,
      Name: 'images/cat-pet-animal-domestic-104827.jpeg'
    }
  },
  MaxLabels: 10,
  MinConfidence: 80
}

// call Rekognition
rek.detectLabels(imageParams, (err, data) => {
  console.log(data)              // print raw results
  support.printRekResults(data)  // print formatted results
})
$$.clear()

The call to rekogntion returns JSON formatted data, we have provided an additional support fuction that formats the output. Notice that each label comes with an associated confidence level (a number between 0 and 100). It is important when using AI services that you interpret this confidence level. For example it is usually not a good idea to classify an image with a less than 90% confidence level. This does of course depend on the business context.


## 2 Face Detection
Here is a picture of some happy go getting executives at a business meeting!

![Meeting](../data/images/Business_Meeting.jpg "Meeting")

AWS Rekognition has built in facial recognition capabilities. The code below wil feed this image to Rekognition and ask it to detect faces in the image.

In [ ]:
// load the AWS SDK
AWS = require('aws-sdk')
support = require('/home/dev/support')

// create a Rekogntion object to call
rek = new AWS.Rekognition()

// configure parameters
imageParams = {
  Image: {
    S3Object: {
      Bucket: process.env.MY_BUCKET_NAME,
      Name: 'images/Business_Meeting.jpg'
    }
  }
}

// call the detect faces API
rek.detectFaces(imageParams, (err, data) => {
  console.log(data)  // output the raw JSON
  support.boxImage(data.FaceDetails, '../data/images/Business_Meeting.jpg', '../data/images/Business_Meeting_boxed.png', err => {
    // draw the boxed image
    $$.png(require('fs').readFileSync('../data/images/Business_Meeting_boxed.png').toString('base64'));
  })
})
$$.clear()

In the example, we have used some supporting code to draw rectangles over the image based on the results returned by Rekognition. This is in the function `boxImage` in `support.js` in the root of this repository.


# Celebrity Detection
Rekognition also has build in APIs for detecting celebreties. The image below was take at the 2007 Oscars.

![Celebs](../data/images/2007_iconic_acting_lewis_Cotillard_Swinton_Bardem.jpg "Celebs")

Now it's your turn to try calling Rekognition. Following the pattern from the last two examples, configure the parameters and figure out the API to call. We have provided you with a function `boxCelebs` in the support library that will draw boxes around the faces as before, see if you can get the sample below to work. The documentation for the Rekognition API can be found here: https://docs.aws.amazon.com/AWSJavaScriptSDK/latest/AWS/Rekognition.html

In [ ]:
// load the AWS SDK
AWS = require('aws-sdk')
support = require('/home/dev/support')

// create a Rekogntion object to call
rek = new AWS.Rekognition()

// configure parameters
imageParams = {
  Image: {
    S3Object: {
      Bucket: process.env.MY_BUCKET_NAME,
      Name: 'images/2007_iconic_acting_lewis_Cotillard_Swinton_Bardem.jpg'
    }
  },
}


/* TODO: fill in the rekognition API to detect celebrties */ (imageParams, (err, data) => {
  support.boxCelebs(data.CelebrityFaces, '../data/images/2007_iconic_acting_lewis_Cotillard_Swinton_Bardem.jpg', '../data/images/celebs_boxed.png', err => {
    $$.png(require('fs').readFileSync('../data/images/celebs_boxed.png').toString('base64'));
  })
})

$$.clear()

# Face Comparison
Rekogntion also provides the abiltity to seach for faces within images. This is a powerful capability. Below we have two images. One of Patrick Stewart as Captian Jean Luc Picard and aother with him hanging out with his celebrity buddies at an awards ceremony.

<img src='../data/images/picard.jpg' width='300'>&nbsp;&nbsp;<img src='../data/images/Sci-Tech-Oscars-group-photo.jpg' width='600'>

We will use Reckognition to see if we can detect Patrick at the awards ceremony given the picture of him in Star Treck. As before see if you can plug in the correct API from the Rekconition documentation which can be found here: https://docs.aws.amazon.com/AWSJavaScriptSDK/latest/AWS/Rekognition.html

In [ ]:
// load the AWS SDK
AWS = require('aws-sdk')
support = require('/home/dev/support')

// create a Rekogntion object to call
rek = new AWS.Rekognition()

// configure parameters
imageParams = {
  SimilarityThreshold: 90, 
  SourceImage: {
    S3Object: {
      Bucket: process.env.MY_BUCKET_NAME,
      Name: "images/picard.jpg"
    }
  }, 
  TargetImage: {
    S3Object: {
     Bucket: process.env.MY_BUCKET_NAME,
     Name: "images/Sci-Tech-Oscars-group-photo.jpg"
    }
  }
}

/* TODO: fill in the rekognition API to detect celebrties */(imageParams, (err, data) => {
support.boxImage(data.FaceMatches, '../data/images/Sci-Tech-Oscars-group-photo.jpg', '../data/images/sci-oscars_boxed.png', err => {
    $$.png(require('fs').readFileSync('../data/images/sci-oscars_boxed.png').toString('base64'));
  })
})

$$.clear()

Congratulations! You have completed Step 5. Let's move onto step 6 and deploy the rest of our system!